In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from dataset_path import output_path
path = output_path

In [3]:
df = pd.read_csv('df_master_3.csv')

In [4]:
df.rename(columns={'race_recoded': 'race', 'triage_temperature_recoded_2': 'triage_temperature','triage_heartrate_recoded': 'triage_heartrate',
                  'triage_resprate_recoded': 'triage_resprate','triage_o2sat_recoded': 'triage_o2sat','triage_sbp_recoded': 'triage_sbp',
                  'triage_dbp_recoded': 'triage_dbp','triage_pain_recoded': 'triage_pain'}, inplace=True)

In [5]:
variable_selected = ['age','MAP','triage_acuity','triage_heartrate','triage_temperature','triage_resprate','triage_o2sat','n_ed_365d']
outcome = "outcome_hospitalization"


In [6]:
df[variable_selected].describe()

,age,MAP,triage_acuity,triage_heartrate,triage_temperature,triage_resprate,triage_o2sat,n_ed_365d
count,427795.000000,427795.000000,427795.000000,427795.000000,427795.000000,427795.000000,427795.000000,427795.000000
mean,52.657948,96.705046,2.655947,85.037257,98.030727,17.542716,98.391490,1.388490
std,20.575198,15.194541,0.681211,17.579817,1.841058,2.314970,2.088451,4.105288
min,18.000000,30.000000,1.000000,30.000000,53.200000,10.000000,60.000000,0.000000
25%,34.000000,86.666667,2.000000,72.000000,97.600000,16.000000,97.000000,0.000000
50%,53.000000,96.000000,3.000000,84.000000,98.000000,18.000000,99.000000,0.000000
75%,68.000000,106.000000,3.000000,96.000000,98.500000,18.000000,100.000000,1.000000
max,103.000000,215.000000,5.000000,250.000000,108.140000,60.000000,100.000000,108.000000


In [7]:
X_train = df[variable_selected].copy()
Y_train = df[outcome].copy()



In [18]:
df[variable_selected+ [outcome]].to_csv(os.path.join(path, 'data.csv'), index=False)

In [19]:
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
pre_process = ColumnTransformer(remainder ='passthrough',
                               transformers =[('scale_data', numeric_transformer,variable_selected)])
pre_process.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('scale_data',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['age', 'MAP', 'triage_acuity',
                                  'triage_heartrate', 'triage_temperature',
                                  'triage_resprate', 'triage_o2sat',
                                  'n_ed_365d'])])

In [20]:
rf= RandomForestClassifier()

model = rf.fit(pre_process.transform(X_train),Y_train)

In [21]:
import pickle
# Save the model to disk
filename = './hosptab/rf.sav'
pickle.dump(model, open(filename, 'wb'))

# Save the pipeline to disk
filename = './hosptab/pre_process.sav'
pickle.dump(pre_process, open(filename, 'wb'))

In [22]:
def Hospitalization_Prediction(_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8):

    input_data = np.column_stack([_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8])
    X = pd.DataFrame(input_data,columns=['age','MAP','triage_acuity','triage_heartrate','triage_temperature','triage_resprate','triage_o2sat','n_ed_365d'])
    result = model.predict_proba(pre_process.transform(X))
    return [probability[1] for probability in result]

In [13]:
from tabpy.tabpy_tools.client import Client
client = Client('http://localhost:9004/')

In [23]:

client.deploy('Hospitalization_Prediction',
Hospitalization_Prediction,
'Returns prediction of probability of Hospitalization.',override=True)

In [24]:
Hospitalization_Prediction(35,93.33,3,80,98,31,100,1)


[0.37]

In [136]:
Hospitalization_Prediction([52,25], [77.33,67],[3,2],[70,78],[98.4,104],[16,10],[97,99],[0,2])

[0.84, 0.6]

In [9]:
import tabpy_client

In [10]:
# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')

In [11]:
connection.deploy('Hospitalization_Prediction',
Hospitalization_Prediction,
'Returns prediction of probability of Hospitalization.',override = True)

NameError: name 'Hospitalization_Prediction' is not defined

In [15]:
client.remove('DiagnosticsDemo')